 # How to use the BRON arangodb
 
 You can view the public version of BRON with the arangodb web interface at [http:bron.alfa.csail.mit.edu:8529] with `username` `guest` and `password` `guest`.

In [11]:
# Change working directory
import os
cwd = os.path.split(os.getcwd())
if cwd[-1] == 'tutorials':
    os.chdir('..')


In [12]:
import arango

BRON_SERVER_IP = 'bron.alfa.csail.mit.edu'
BRON_USERNAME = 'guest'
BRON_PASSWORD = 'guest'
DB = 'BRON'

## Example connect to BRON arangodb and perform AQL query

In [13]:
%%time
query = """
FOR c IN tactic
    FILTER c.original_id == "TA0006"
    RETURN c
"""
client = arango.ArangoClient(hosts=f"http://{BRON_SERVER_IP}:8529")
db = client.db(DB, username=BRON_USERNAME, password=BRON_PASSWORD, auth_method="basic")
assert db.aql.validate(query)
cursor = db.aql.execute(query)
for c in cursor:
    print(c)


{'_key': 'tactic_00003', '_id': 'tactic/tactic_00003', '_rev': '_dzuWJ3---A', 'original_id': 'TA0006', 'datatype': 'tactic', 'name': 'credential-access', 'metadata': {'description': 'The adversary is trying to steal account names and passwords.\n\nCredential Access consists of techniques for stealing credentials like account names and passwords. Techniques used to get credentials include keylogging or credential dumping. Using legitimate credentials can give adversaries access to systems, make them harder to detect, and provide the opportunity to create more accounts to help achieve their goals.', 'short_description': 'The adversary is trying to steal account names and passwords.'}}
CPU times: user 8.98 ms, sys: 536 µs, total: 9.52 ms
Wall time: 14.5 ms


In [14]:
%%time
query_template = """
FOR c IN capec
    FILTER c.original_id == "{}"
    FOR v IN 1..1 ANY c {}
        RETURN v
"""
capec = "117"
edge_collections = ("TechniqueCapec", "CapecCwe")
for edge_collection in edge_collections:
    query = query_template.format(capec, edge_collection)
    assert db.aql.validate(query)
    print(query)
    cursor = db.aql.execute(query)
    results = {_['original_id'] for _ in cursor}
    print(f'{len(results)} result for CAPEC {capec} in {edge_collection}: {results}')



FOR c IN capec
    FILTER c.original_id == "117"
    FOR v IN 1..1 ANY c TechniqueCapec
        RETURN v

1 result for CAPEC 117 in TechniqueCapec: {'T1020.001'}

FOR c IN capec
    FILTER c.original_id == "117"
    FOR v IN 1..1 ANY c CapecCwe
        RETURN v

1 result for CAPEC 117 in CapecCwe: {'319'}
CPU times: user 12.5 ms, sys: 3.63 ms, total: 16.1 ms
Wall time: 22.1 ms
